In [2]:
%load_ext autoreload

In [25]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [26]:
#config_file = "configs/vgg_ssd300_voc0712_tdt4265_server.yaml"
#config_file = "configs/mnist_tdt4265_server_experimental.yaml"
config_file = "configs/train_rdd2020_server.yaml"
# config_file = "configs/train_rdd2020_server_experimental.yaml"
cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-04-19 16:52:44,967 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-19 16:52:44,967 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-19 16:52:44,967 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-19 16:52:44,967 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-19 16:52:44,967 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-19 16:52:44,967 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-19 16:52:44,967 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-19 16:52:44,967 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-19 16:52:44,975 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'res_net'
        PRETRAINED: True
        OUT_CHANNELS: [128, 256, 512, 512, 256, 128]
        INPUT_CHANNELS: 3
INPUT:
    IMAGE_SIZE: [300, 300]
DATASETS:
 

In [27]:
!pwd 
model = start_train(cfg)

/notebooks/TDT4265-Project/assignment4/SSD
Detector initialized. Total Number of params:  21.62M
Backbone number of parameters: 20.84M
SSD Head number of parameters: 788.2K
SSDDetector(
  (backbone): ResNetBackbone(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

AssertionError: Expected shape: (128, 1, 1), got: torch.Size([128, 2, 2]) at output IDX: 5

In [15]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

2021-04-19 16:49:01,085 SSD INFO: Start evaluating...
2021-04-19 16:49:01,085 SSD INFO: Start evaluating...
2021-04-19 16:49:01,085 SSD INFO: Start evaluating...
2021-04-19 16:49:01,085 SSD INFO: Start evaluating...


NameError: name 'model' is not defined